In [ ]:
import tensorflow as tf    
# from tensorflow.examples.tutorials.mnist import input_data  
import os
import numpy as np
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_string('ckpt_path', '/home/artemis/下载/fashion-mnist-master/CNN',"""存放模型的目录""")

tf.app.flags.DEFINE_string('model_name', 'mnist',"""模型的名称""")
  

def weight_variable(shape):  
    initial = tf.truncated_normal(shape, stddev=0.1)  
    return tf.Variable(initial)  
  

def bias_variable(shape):  
    initial = tf.constant(0.1, shape=shape)  
    return tf.Variable(initial)  
  

def conv2d(x, W):  
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')  
  

def max_pool_2x2(x):  
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  
#模型文件所在的文件夹，是否存在，如果不存在，则创建文件夹
ckpt = tf.train.latest_checkpoint(FLAGS.ckpt_path)
if not ckpt:
    if not os.path.exists(FLAGS.ckpt_path):
        os.mkdir(FLAGS.ckpt_path)

# mnist = input_data.read_data_sets("data/fashion", one_hot=True)  
import numpy as np


class Dataset(object):
    def __init__(self, dtype='uint8', is_row_iamge=False):
        '''数据集
        
        Args:
            dtype: uint8 或 float32，uint8：每个像素值的范围是[0, 255];float32像素值范围是[0., 1.]
            is_row_image: 是否将3维图片展开成1维
        '''
        images = np.fromfile('./images/test_image.bin', dtype=np.uint8).reshape(-1, 28, 28, 1)
        print(images.shape)
        if dtype == 'uint8':
            self.images = images
        else:
            images = images.astype(np.float32) / 255.
            self.images = images
        if is_row_iamge:
            self.images = images.reshape([-1, 784])
        self.num_of_images = 6500
        self.offset = 0
        print('共6500张图片')

    def next_batch(self, batch_size=50):
        # 返回False表示以及没有样本
        # 注意：最后一个批次可能不足batch_size 所以推荐选择6500可以整除的batch_size
        if (self.offset + batch_size) <= self.num_of_images:
            self.offset += batch_size
            return self.images[self.offset-batch_size : self.offset]
        elif self.offset < self.num_of_images:
            return self.images[self.offset : ]
        else:
            False

  
sess = tf.InteractiveSession()  
  
x = tf.placeholder("float", shape=[None, 784])  
y_ = tf.placeholder("float", shape=[None, 10])  
  
W = tf.Variable(tf.zeros([784, 10]))  
b = tf.Variable(tf.zeros([10]))  
  
W_conv1 = weight_variable([5, 5, 1, 32])  
b_conv1 = bias_variable([32])  
  
x_image = tf.reshape(x, [-1, 28, 28, 1])  
  
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)  
h_pool1 = max_pool_2x2(h_conv1)  
  
W_conv2 = weight_variable([5, 5, 32, 64])  
b_conv2 = bias_variable([64])  
  
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)  
h_pool2 = max_pool_2x2(h_conv2)  
  
# Now image size is reduced to 7*7  
W_fc1 = weight_variable([7 * 7 * 64, 1024])  
b_fc1 = bias_variable([1024])  
  
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])  
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)  
  
keep_prob = tf.placeholder("float")  
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)  
  
W_fc2 = weight_variable([1024, 10])  
b_fc2 = bias_variable([10])  
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)  
  
  
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))  
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)  
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))  
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))  
saver=tf.train.Saver(max_to_keep=2)
sess.run(tf.initialize_all_variables()) 

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(FLAGS.ckpt_path,sess.graph)
    
    ckpt = tf.train.latest_checkpoint(FLAGS.ckpt_path)
    step=0
    if ckpt:
        saver.restore(sess=sess,save_path=ckpt)
        step = int(ckpt[len(os.path.join(FLAGS.ckpt_path, FLAGS.model_name)) + 1:])
    images = Dataset()   
    Y = np.zeros(6500)   
    for j in range(130):  
        b_img = images.next_batch()   
        b_img = b_img.reshape((50, 784))   
        Ytemp = y_conv.eval(feed_dict={x: b_img, keep_prob: 1.0})  
        for i in range(50):
            Y[j*50+i] = np.argmax(Ytemp[i])
    fp = open("zhaokaiyan.txt", "w+")
    for i in range(6500):
        fp.write(str(int(Y[i]))+"\n")
    fp.close()


    # for i in range(20000):  
    #     batch = mnist.train.next_batch(50)  
    #     if i % 100 == 0:  
    #         train_accuracy = accuracy.eval(feed_dict={  
    #             x: batch[0], y_: batch[1], keep_prob: 1.0})  
    #         print("step %d, training accuracy %.3f" % (i, train_accuracy)) 
    #         ckptname=os.path.join(FLAGS.ckpt_path, FLAGS.model_name)
    #         saver.save(sess,ckptname,global_step=i)
    #     train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})  
    
    # print("Training finished")  
    
    # print("test accuracy %.3f" % accuracy.eval(feed_dict={  
    #     x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})) 